# IMEC2001 Herramientas Computacionales 
## Semana 8: Transformada Rápida de Fourier (FFT)
### Clase 15: FFT

Universidad de los Andes — Mayo 24 y 26, 2023.

---

## TABLA DE CONTENIDO

### Sección 1: FFT [→](#section1)
- 1.1. Introducción
- 1.2. Cargar Librerías
- 1.3. Ejemplo 1: Datos Medidos
    - 1.3.1. Señal 1
    - 1.3.2. Señal 4
    - 1.3.3. Señal 5
- 1.4. Ejemplo 2: Audio

### Sección 2: Sistemas Dinámicos [→](#section2)
- 2.1. Introducción
- 2.2. Ejemplo 1: Péndulo Simple
- 2.3. Ejemplo 2: Péndulo Simple + Fricción + Fuerza Externa
___

<a id="section1"></a>
# Sección 1: FFT

## 1.1. Introducción

La onda de una señal en el tiempo se puede describir a partir de los siguientes parámetros:
- Amplitud $A_m$.
- Frecuencia angular $\omega$.
- Ángulo de fase $\phi$.

Así, la ecuación que la describe es:

$$
s(t) = A_m \cdot \sin (\omega t + \phi)
$$

Al ver gráficamente el comportamiento de la señal, es más sencillo entender su **periodo** $T$ que la frecuencia angular $\omega$. Luego, tenemos la siguiente relación:

$$
\omega = \frac{2 \pi}{T}
$$

<img src='./img/señal.png' width='500' height='500'/>

Pero, qué sucede si tenemos varias ondas juntas en la misma señal en el tiempo, ¿cómo las podemos describir?

<img src='./img/señal_tiempo.png' width='500' height='500'/>

Aquí entra en escena la **Transformada Discreta de Fourier** (DFT, por sus siglas en inglés). Este es un algoritmo que transforma una función matemática en otra, obteniendo una representación en el dominio de la frecuencia, siendo la función original una función en el dominio del tiempo.

La representación en el dominio de la frecuencia es la descripción descompuesta de la onda de la señal original, es decir, descompone una señal en sus componentes espectrales individuales y así proporciona información sobre su composición. 

La representación matemática de la señal original es:

$$
f(t) = A_0 + A_1 \sin(\omega t + \phi_1) + A_2 \sin(2 \omega t + \phi_2) + ... + A_n \sin(n \omega t + \phi_n)
$$

Para llegar a esta solución, debemos conocer:
- El periodo $T$ de la señal.
- La cantidad de datos $N$ requeridos para comlpetar cada periodo.
- La frecuencia de muestreo (también llamado el *paso de frecuencias* o *resolución de frecuencias*), determinado como $\Delta f = 1/T$.
- La frecuencia circular $\omega = 2 \pi / T$.

La **Transformada Rápida de Fourier** (FFT, por sus siglas en inglés) es un algoritmo que reduce considerablemente el tiempo de cálculo de la DFT.

Aprovecharemos la librería `scipy.fft` para comprender de forma práctica y eficiente qué es la FFT y cómo la podemos aplicar al trabajar con señales en el tiempo.

<div class='alert alert-block alert-info'> 

<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información en la documentación oficial de la librería `scipy.io.wavfile` dando clic [aquí](https://docs.scipy.org/doc/scipy/tutorial/fft.html).
</div>

¡Empecemos!

## 1.2. Cargar Librerías

In [ ]:
# Datos y Gráficas
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import integrate

# FFT
from scipy import fft
from scipy.fftpack import fftfreq

## 1.3. Ejemplo 1: Datos Medidos

Tenemos cinco señales diferentes en el tiempo y queremos caracterizar cada una de ellas de forma independiente utilizando la FFT mediante la librería `scipy.fft`.

In [ ]:
df = pd.read_excel(io='./data/señales.xlsx', 
                   sheet_name='Hoja1') # ./ es pwd()

df.head()

### 1.3.1. Señal 1

In [ ]:
def formato_grafica(titulo, ejex, ejey, leyenda=False, xlim=[None, None], ylim=[None, None]):
    plt.rcParams['axes.axisbelow'] = True

    plt.title(titulo, fontsize=15)
    plt.ylabel(ejey, fontsize=13)
    plt.xlabel(ejex, fontsize=13)

    plt.tick_params(direction='out', length=5, width=0.75, grid_alpha=0.3)
    plt.xticks(rotation=0)
    plt.minorticks_on()
    plt.ylim(ylim[0], ylim[1])
    plt.xlim(xlim[0], xlim[1])
    plt.grid(True)
    plt.grid(visible=True, which='major', color='grey', linestyle='-')
    plt.grid(visible=True, which='minor', color='lightgrey', linestyle='-', alpha=0.2)
    
    if leyenda == True:
        plt.legend(loc='best')
    
    plt.tight_layout;

In [ ]:
hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(df['tiempo_ms'], df['f1'], linestyle='--', marker='o', markerfacecolor='white', markeredgecolor='dodgerblue', ms=10)

formato_grafica(titulo='Respuesta Función 1', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                leyenda=False)

Recordemos que para poder implementar la FFT, debemos conocer:
- El periodo $T$ de la señal.
- La cantidad de datos $N$ requeridos para comlpetar cada periodo.
- La frecuencia de muestreo (también llamado el *paso de frecuencias* o *resolución de frecuencias*), determinado como $\Delta f = 1/T$.
- La frecuencia circular $\omega = 2 \pi / T$.

In [ ]:
# Periodo
T = 18 # ms

# Cantidad de datos
N = len(df['f1'])

# Frecuencia de muestreo
def frec_muestreo(T):
    return 1/T

delta_f = frec_muestreo(T) # Hz

# Frecuencia Circular
def freq_circular(T):
    return 2*np.pi / (T/1000)

omega = freq_circular(T) # rad/s
omega

Para emplear la librería `scipy.fft` se requiere conocer (i.) la señal y (ii.) la frecuencia de muestreo. La sintaxis es:

In [ ]:
import scipy

In [ ]:
# PASO 1. Estimar la FFT
señal_fft = fft.fft(df['f1'].values)
señal_fft

In [ ]:
# PASO 2. Estimar las frecuencias
frecs = fftfreq(N, delta_f)
frecs

In [ ]:
# PASO 3. Calcular la amplitud
An = (2/N) * abs(señal_fft)
An

In [ ]:
hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(df['tiempo_ms'], df['f1'], linestyle='--', marker='o', markerfacecolor='white', markeredgecolor='dodgerblue', ms=10)

formato_grafica(titulo='Respuesta Función 1', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                leyenda=False)

hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(frecs, An, linestyle='-', color='peru', linewidth=1, marker='o', markerfacecolor='white', markeredgecolor='orange', ms=10)

formato_grafica(titulo='Respuesta Función 1', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                xlim=(0, None),
                leyenda=False)

In [ ]:
# Amplitudes
n_picos = 2

max_An = np.unique(An[np.argpartition(An, -n_picos*2)[-n_picos*2:]])
max_An

In [ ]:
# Ángulos de Fase
phi = np.angle(señal_fft)

# Ángulos de amplitudes
n_picos = 2

max_phi = np.unique(phi[np.argpartition(phi, -n_picos)[-n_picos:]])
max_phi

Conocidos estos valores, podemos obtener la representación matemática de la DFT; esta es:

$$
\boxed{ f(t) = 3.66 \sin(2 \omega t) + 1.83 \sin(4 \omega t) }
$$

### 1.3.2. Señal 4

In [ ]:
hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(df['tiempo_ms'], df['f4'], linestyle='--', marker='o', markerfacecolor='white', markeredgecolor='dodgerblue', ms=10)

formato_grafica(titulo='Respuesta Función 4', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                leyenda=False)

In [ ]:
# Periodo
T = 36 # ms

# Cantidad de datos
N = len(df['f4'])

# Frecuencia de muestreo
delta_f = frec_muestreo(T) # Hz

# Frecuencia Circular
omega = freq_circular(T) # rad/s
omega

In [ ]:
# PASO 1. Estimar la FFT
señal_fft = fft.fft(df['f4'].values)
señal_fft

In [ ]:
# PASO 2. Estimar las frecuencias
frecs = fftfreq(N, delta_f)
frecs

In [ ]:
# PASO 3. Calcular la amplitud
An = (2/N) * abs(señal_fft)
An

In [ ]:
hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(df['tiempo_ms'], df['f4'], linestyle='--', marker='o', markerfacecolor='white', markeredgecolor='dodgerblue', ms=10)

formato_grafica(titulo='Respuesta Función 4', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                leyenda=False)

hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(frecs, An, linestyle='-', color='peru', linewidth=1, marker='o', markerfacecolor='white', markeredgecolor='orange', ms=10)

formato_grafica(titulo='Respuesta Función 4', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                xlim=(0, None),
                leyenda=False)

In [ ]:
# Amplitudes
n_picos = 3

max_An = np.unique(An[np.argpartition(An, -n_picos*2)[-n_picos*2:]])
max_An

In [ ]:
# Ángulos de Fase
phi = np.angle(señal_fft)

# Ángulos de amplitudes
n_picos = 3

max_phi = np.unique(phi[np.argpartition(phi, -n_picos)[-n_picos:]])
max_phi

Conocidos estos valores, podemos obtener la representación matemática de la DFT; esta es:

$$
\boxed{ f(t) = 9.88 \sin(\omega t + 3.12) + 3.98 \sin(5 \omega t + 2.36) + 2.08 \sin(6 \omega t + 1.93) }
$$

### 1.3.3. Señal 5

In [ ]:
hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(df['tiempo_ms'], df['f5'], linestyle='--', marker='o', markerfacecolor='white', markeredgecolor='dodgerblue', ms=10)

formato_grafica(titulo='Respuesta Función 5', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                leyenda=False)

In [ ]:
# Periodo
T = 36 # ms

# Cantidad de datos
N = len(df['f5'])

# Frecuencia de muestreo
delta_f = frec_muestreo(T) # Hz

# Frecuencia Circular
omega = freq_circular(T) # rad/s
omega

In [ ]:
# PASO 1. Estimar la FFT
señal_fft = fft.fft(df['f5'].values)
señal_fft

In [ ]:
# PASO 2. Estimar las frecuencias
frecs = fftfreq(N, delta_f)
frecs

In [ ]:
# PASO 3. Calcular la amplitud
An = (2/N) * abs(señal_fft)
An

In [ ]:
hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(df['tiempo_ms'], df['f5'], linestyle='--', marker='o', markerfacecolor='white', markeredgecolor='dodgerblue', ms=10)

formato_grafica(titulo='Respuesta Función 5', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                leyenda=False)

hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(frecs, An, linestyle='-', color='peru', linewidth=1, marker='o', markerfacecolor='white', markeredgecolor='orange', ms=10)

formato_grafica(titulo='Respuesta Función 5', 
                ejex='Tiempo (ms)', 
                ejey='Voltaje (V)',
                xlim=(0, None),
                leyenda=False)

In [ ]:
# Amplitudes
n_picos = 2

max_An = np.unique(An[np.argpartition(An, -n_picos*2)[-n_picos*2:]])
max_An

In [ ]:
# Ángulos de Fase
phi = np.angle(señal_fft)

# Ángulos de amplitudes
n_picos = 2

max_phi = np.unique(phi[np.argpartition(phi, -n_picos)[-n_picos:]])
max_phi

Conocidos estos valores, podemos obtener la representación matemática de la DFT; esta es:

$$
\boxed{ f(t) = 7.52 \sin(2 \omega t + 3.14) + 5.06 \sin(3 \omega t + 3.12) }
$$

## 1.4. Ejemplo 2: Audio

Tenemos cuatro señales de audio independientes y una quinta que las combina todas. Vamos a ver cómo utilizando la librería `scipy.fft` podemos obtener la caracterización de cada una de las señales que componen el audio completo.

Dado que los audio son en formato WAV, debemos utilizar la misma librería `scipy.io.wavfile` para reconocerlos en Python.

<div class="alert alert-block alert-success">
    
**Nota:** Ejercicio tomado de MIT 18.S191 (2020), Introduction to Computational Thinking: Lecture 26 -- Discrete Fourier Transform. [online]. Disponible [aquí](https://www.youtube.com/watch?v=g8RkArhtCc4) y [aquí](https://github.com/mitmath/18S191/blob/Fall20/lecture_notebooks/week13/dft.jl).
</div>

<div class='alert alert-block alert-info'> 

<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información en la documentación oficial de la librería `scipy.io.wavfile` dando clic [aquí](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.read.html).
</div>

In [ ]:
from scipy.io import wavfile

_, data1 = wavfile.read('./data/Sound1.wav')
_, data2 = wavfile.read('./data/Sound2.wav')
_, data3 = wavfile.read('./data/Sound3.wav')
_, data4 = wavfile.read('./data/Sound4.wav')
_, data1234 = wavfile.read('./data/Sound1234.wav')

In [ ]:
hor = 8
ver = 5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(data1[:,0], linestyle='-', linewidth=0.5, color='dodgerblue', label='Izquierdo')
plt.plot(data1[:,1], linestyle='-', linewidth=0.5, color='orange', label='Derecho')

formato_grafica(titulo='', 
                ejex='', 
                ejey='',
                leyenda=True)

In [ ]:
# Visualización de la señal combinada
hor = 8
ver = 5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(data1234[:,0][20000:22000], linestyle='-', linewidth=1, color='orchid', label='Sonido Combinado')

formato_grafica(titulo='', 
                ejex='', 
                ejey='',
                leyenda=True)

In [ ]:
# Visualización de las señales
hor = 8
ver = 5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(data1[:,0][20000:22000], linestyle='-', linewidth=1, color='dodgerblue', label='Sonido 1')
plt.plot(data2[:,0][20000:22000], linestyle='-', linewidth=1, color='orange', label='Sonido 2')
plt.plot(data3[:,0][20000:22000], linestyle='-', linewidth=1, color='mediumseagreen', label='Sonido 3')
plt.plot(data4[:,0][20000:22000], linestyle='-', linewidth=1, color='tomato', label='Sonido 4')

formato_grafica(titulo='', 
                ejex='', 
                ejey='',
                leyenda=True)

In [ ]:
# PASO 1. Estimar la FFT
señal_fft1 = fft.fft(data1[:,0])
señal_fft2 = fft.fft(data2[:,0])
señal_fft3 = fft.fft(data3[:,0])
señal_fft4 = fft.fft(data4[:,0])

In [ ]:
# FFT señal combinada
señal_fft1234 = fft.fft(data1234[:,0])

# Gráfica
hor = 8
ver = 5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(np.abs(señal_fft1234[0:1000]), linestyle='-', color='orchid', linewidth=1)

formato_grafica(titulo='FFT', 
                ejex='Frecuencia (Hz)', 
                ejey='Amplitud',
                xlim=(0, None),
                leyenda=False)

In [ ]:
hor = 8
ver = 5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(np.abs(señal_fft1[0:1000]), linestyle='-', color='dodgerblue', linewidth=1)
plt.plot(np.abs(señal_fft2[0:1000]), linestyle='-', color='orange', linewidth=1)
plt.plot(np.abs(señal_fft3[0:1000]), linestyle='-', color='mediumseagreen', linewidth=1)
plt.plot(np.abs(señal_fft4[0:1000]), linestyle='-', color='tomato', linewidth=1)

formato_grafica(titulo='FFT', 
                ejex='Frecuencia (Hz)', 
                ejey='Amplitud',
                xlim=(0, None),
                leyenda=False)

De lo anterior, notemos que **la FFT extrae los componentes de frecuencia de cada sonido**, en donde podemos asociar las notas musicales a los picos de frecuencia de cada señal, siendo los picos más pequeños los armónicos (múltiplos de números enteros de esa frecuencia principal).

En este caso, decimos \"frecuencia\", a menos que cambiemos la escala del eje x, no serán ciclos por unidad de tiempo, sino ciclos por $T$ unidades de tiempo, donde $T$ es la duración del audio.

La FFT solo ve una lista de valores de muestra, no tiene forma de saber cuántas muestras por segundo hay, o si los segundos son la unidad de tiempo que nos interesa, por lo que la única unidad de tiempo sería la duración del audio.

Comentarios de MIT 18.S191 (2020).

<img src='./img/notas.jpeg' width='700' height='700'/>

<a id="section2"></a>
# Sección 2: Sistemas Dinámicos

## 2.1. Introducción

Tengamos presentes las siguientes equivalencias:

#### Movimiento Rectilíneo
$$
\begin{cases}
  v = \frac{dx}{dt} = \dot{x} \\ 
  \\
  a = \frac{dv}{dt} = \frac{d}{dt} \left( \frac{dx}{dt} \right) = \frac{d^2x}{dt^2} = \ddot{x} \\
\end{cases} 
$$

#### Movimiento Rotacional
$$
\begin{cases}
  \omega = \frac{d \theta}{dt} = \dot{\theta} \\ 
  \\
  \alpha = \frac{d \omega}{dt} = \frac{d}{dt} \left( \frac{d \theta}{dt} \right) = \frac{d^2 \theta}{dt^2} = \ddot{\theta} \\
\end{cases} 
$$

#### Generalización 2da Ley de Newton
$$
\begin{cases}
  F = ma \rightarrow F = m \ddot{x} & (1) \\ 
  \\
  T = I \alpha \rightarrow T = I \ddot{\theta} & (2) \\
\end{cases} 
$$

Siendo $F$ la fuerza, $m$ la masa, $a$ la aceleración, $T$ el torque, $I$ el momento de inercia (en nuestros casos de estudio, $I = m L^2$, siendo $L$ la longitud al punto de pivote del giro) y $\theta$ el ángulo de giro.

## 2.2. Ejemplo 1: Péndulo Simple

<img src='./img/pendulum1.png' width='300' height='300'/>

Partimos de la relación:

$$
T = I \ddot{\theta}
$$

El único componente de la masa que causa un torque en el cuerpo es:

$$
m g \sin \theta
$$

Luego:

$$
\boxed{ \ddot{\theta} + \frac{g}{L} \sin \theta = 0 }
$$

<div class='alert alert-block alert-info'> 

<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información del ejercicio dando clic [aquí](https://www.youtube.com/watch?v=8VJ1CJ55Np0) y [aquí](https://www.youtube.com/watch?v=xBBXlQ7CmFc).
</div>

In [ ]:
# PASO 1. Definir la función
def pendulo_simple(variables, t, g, L):
    # Variables
    theta = variables[0]
    dtheta_dt = variables[1]
    
    # Ecuaciones
    d2theta_dt2 = -(g/L) * np.sin(np.deg2rad(theta))
    
    return [dtheta_dt, d2theta_dt2]

pendulo_simple

In [ ]:
# PASO 2. Condiciones iniciales
condiciones_iniciales = [30, 0] # [x0, dx_dt0]
condiciones_iniciales

In [ ]:
# PASO 3. Puntos de tiempo
start = 0
stop = 200
num = 1000

tiempo = np.linspace(start, stop, num)
tiempo

In [ ]:
# PASO 4. Solcionar ODE
## Constantes
g = 9.81
L = 5.0

## Solución numérica
sol = integrate.odeint(func=pendulo_simple, 
                       y0=condiciones_iniciales, 
                       t=tiempo,
                       args=(g, L))

sol

In [ ]:
# Gráfica
hor = 8
ver = 5
fig1 = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(tiempo, sol[:,0], linestyle='-', linewidth=3, color='dodgerblue', label='Solución Numérica')
plt.scatter(tiempo[170], sol[:,0][170], color='orange', zorder=5)

formato_grafica(titulo='Comportamiento ODE', 
                ejex='Tiempo ($t$)', 
                ejey='$theta(t)$',
                leyenda=False)

In [ ]:
# FFT
# Periodo
T = tiempo[170] # ms

# Cantidad de datos
N = len(sol[:,1])

# Frecuencia de muestreo
deltaf = frec_muestreo(T) # Hz
print(f'Frecuencia Muestreo: {np.round(delta_f, 2)} Hz')

# Frecuencia Circular
omega = freq_circular(T) # rad/s
print(f'Frecuencia Circular: {np.round(omega, 2)} rad/s')

In [ ]:
# PASO 1. Estimar la FFT
señal_fft = fft.fft(sol[:,0])
señal_fft

In [ ]:
# PASO 2. Estimar las frecuencias
frecs = fftfreq(N, delta_f)
frecs

In [ ]:
# PASO 3. Calcular la amplitud
An = (2/N) * abs(señal_fft)
An

In [ ]:
hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(tiempo, sol[:,0], linestyle='-', linewidth=3, color='dodgerblue', label='Solución Numérica')

formato_grafica(titulo='', 
                ejex='Tiempo (t)', 
                ejey='Magnitud',
                leyenda=False)

hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(frecs[0:100], An[0:100], linestyle='-', color='peru', linewidth=1, marker='o', markerfacecolor='white', markeredgecolor='orange', ms=10)

formato_grafica(titulo='', 
                ejex='Frecuencia (Hz)', 
                ejey='Amplitud',
                xlim=(0, None),
                leyenda=False)

In [ ]:
# Amplitudes
n_picos = 3

max_An = np.unique(An[np.argpartition(An, -n_picos*2)[-n_picos*2:]])
max_An

In [ ]:
# Ángulos de Fase
phi = np.angle(señal_fft)

# Ángulos de amplitudes
n_picos = 3

max_phi = np.unique(phi[np.argpartition(phi, -n_picos)[-n_picos:]])
max_phi

Conocidos estos valores, podemos obtener la representación matemática de la DFT; para el armónico principal, esta es:

$$
\boxed{ f(t) = 27.89 \sin(3 \omega t + 3.14) }
$$

## 2.3. Ejemplo 2: Péndulo Simple + Fricción + Fuerza Externa

<img src='./img/pendulum2.png' width='800' height='800'/>

Este caso es una extensión al Ejemplo 1, pues:

$$
F = m a
$$

Luego:

$$
F_0 \cos(\omega t) - mg \sin(\theta) - c \frac{ds}{dt} = ma 
$$

Entonces:

$$
F_0 \cos(\omega t) = m L \ddot{\theta} + c L \dot{\theta} + mg \sin(\theta)
$$

Que se puede reescribir como:

$$
\boxed{ \ddot{\theta} + \alpha \dot{\theta} + \sin(\theta) = \gamma \cos(\beta t) }
$$

Donde:

$$
\begin{cases}
  \alpha = \frac{c}{m \omega_0} \\ 
  \\
  \beta = \frac{\omega}{\omega_0} \\
  \\
  \gamma = \frac{F_0}{m L \omega_0^2}
\end{cases} 
$$

<div class='alert alert-block alert-info'> 

<i class='fa fa-info-circle' aria-hidden='true'></i>
Puede obtener más información del ejercicio dando clic [aquí](https://www.youtube.com/watch?v=SZWn7x4g-Vo).
</div>

In [ ]:
# PASO 1. Definir la función
def pendulo_friccion(variables, t, alpha, beta, gamma):
    # Variables
    theta = variables[0]
    dtheta = variables[1]
    
    # Ecuaciones
    d2theta = -alpha*dtheta - np.sin(np.deg2rad(theta)) + gamma*np.cos(np.deg2rad(beta*t))
    
    return [dtheta, d2theta]

pendulo_friccion

In [ ]:
# PASO 2. Condiciones iniciales
condiciones_iniciales = [30, 0] # [x0, dx_dt0]
condiciones_iniciales

In [ ]:
# PASO 3. Puntos de tiempo
start = 0
stop = 1000
num = 1000

tiempo = np.linspace(start, stop, num)
tiempo

In [ ]:
# PASO 4. Solcionar ODE
## Constantes
c = 0.5
m = 2.0
omega = 50.0
beta = 0.98
F = 150
L = 8.0

alpha = c / (m*omega)
gamma = F / (m*L*(omega**2))

## Solución numérica
sol = integrate.odeint(func=pendulo_friccion, 
                       y0=condiciones_iniciales, 
                       t=tiempo,
                       args=(alpha, beta, gamma))

sol

In [ ]:
# Gráfica
hor = 10
ver = 3

# Ángulo
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(tiempo, sol[:,0], linestyle='-', linewidth=2, color='dodgerblue', label='Solución Numérica')

formato_grafica(titulo='', 
                ejex='Tiempo ($t$)', 
                ejey='θ (rad)',
                leyenda=False)

# Velocidad Angular
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(tiempo, sol[:,1], linestyle='-', linewidth=2, color='orange', label='Solución Numérica')
plt.scatter(tiempo[35], sol[:,1][35], color='tomato', zorder=5)

formato_grafica(titulo='', 
                ejex='Tiempo ($t$)', 
                ejey='ω (rad/s)',
                leyenda=False)

In [ ]:
# FFT
# Periodo
T = tiempo[35] # ms

# Cantidad de datos
N = len(sol[:,1])

# Frecuencia de muestreo
deltaf = frec_muestreo(T) # Hz
print(f'Frecuencia Muestreo: {np.round(delta_f, 2)} Hz')

# Frecuencia Circular
omega = freq_circular(T) # rad/s
print(f'Frecuencia Circular: {np.round(omega, 2)} rad/s')

In [ ]:
# PASO 1. Estimar la FFT
señal_fft = fft.fft(sol[:,1])
señal_fft

In [ ]:
# PASO 2. Estimar las frecuencias
frecs = fftfreq(N, delta_f)
frecs

In [ ]:
# PASO 3. Calcular la amplitud
An = (2/N) * abs(señal_fft)
An

In [ ]:
hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(tiempo, sol[:,1], linestyle='-', linewidth=3, color='dodgerblue', label='Solución Numérica')

formato_grafica(titulo='', 
                ejex='Tiempo (t)', 
                ejey='Magnitud',
                leyenda=False)

hor = 8
ver = 2.5
fig = plt.figure(figsize=(hor, ver), dpi=80)

plt.plot(frecs[0:50], An[0:50], linestyle='-', color='peru', linewidth=1, marker='o', markerfacecolor='white', markeredgecolor='orange', ms=10)

formato_grafica(titulo='', 
                ejex='Frecuencia (Hz)', 
                ejey='Amplitud',
                xlim=(0, None),
                leyenda=False)

In [ ]:
# Amplitudes
n_picos = 4

max_An = np.unique(An[np.argpartition(An, -n_picos*2)[-n_picos*2:]])
max_An

In [ ]:
# Ángulos de Fase
phi = np.angle(señal_fft)

# Ángulos de amplitudes
n_picos = 4

max_phi = np.unique(phi[np.argpartition(phi, -n_picos)[-n_picos:]])
max_phi

Conocidos estos valores, podemos obtener la representación matemática de la DFT; para el armónico principal, esta es:

$$
\boxed{ f(t) = 3.13 \sin(22 \omega t + 1.42) }
$$